<a href="https://colab.research.google.com/github/leman-cap13/my_projects/blob/main/Semed_Vurgun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
from google.colab import files
files.upload()


In [ ]:
with open('/content/Semed_Vurgun_seirleri.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
text[:100]

In [ ]:
#Unikal simvollari tapiriq
''.join(sorted(set(text)))

In [ ]:
# lower edirik herfleri
''.join(sorted(set(text.lower())))

In [ ]:
#Tokenlere cevirek
text_vector_layer=tf.keras.layers.TextVectorization(split='character', standardize='lower')
text_vector_layer.adapt([text])
text_vector_layer.get_vocabulary()

In [ ]:
#encode edirik
encode=text_vector_layer([text][0])

In [ ]:
encode

In [ ]:
encode-=2

In [ ]:
# vocabulary size ve dataset size tap
vocab_size=text_vector_layer.vocabulary_size()-2
dataset_size=len(encode)

In [ ]:
print("vocab_size:", vocab_size)
print("max index in encode:", encode.numpy().max())


In [ ]:
dataset_size

In [ ]:
vocab_size

In [ ]:
#make api function
def to_dataset(sequence,length,shuffle=True, seed=None, batch_size=32):
  ds=tf.data.Dataset.from_tensor_slices(sequence)
  ds=ds.window(length+1,shift=1,drop_remainder=True)
  ds=ds.flat_map(lambda window_ds: window_ds.batch(length+1))
  if shuffle:
    ds=ds.shuffle(100_000, seed=seed)
  ds=ds.batch(batch_size)
  return ds.map(lambda window : (window[:,:-1], window[:,1 :])).prefetch(1)



In [ ]:
#train test ayir
length=100
tf.random.set_seed(42)
train_data=to_dataset(encode[:100_000],shuffle=True,length=length,seed=42)
valid_data=to_dataset(encode[100_000:150_000],shuffle=False,length=length)
test_data=to_dataset(encode[150_000:],shuffle=False,length=length)

In [ ]:
#Model qurmaliyiq
model=tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=64),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

model.fit(train_data, epochs=3, validation_data=valid_data)

In [ ]:
#Semed vurgun model
Semed_Vurgun_model=tf.keras.Sequential([
    text_vector_layer,
    tf.keras.layers.Lambda(lambda x: x-2),
    model
])




In [ ]:
def next_char(text, temperature=1.0):
    text = tf.constant([text])  # Daxil edilən mətni Tensor'a çeviririk
    y_proba = Semed_Vurgun_model.predict(text)[0, -1:]  # Modeli işlədərək növbəti simvolu alırıq
    rescaled_logits = tf.math.log(y_proba) / temperature  # Temperatur tənzimlənməsi
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]  # Növbəti simvolu təxmin edirik
    return text_vector_layer.get_vocabulary()[char_id + 2]  # Təsadüfi simvolu seçirik

# Şeiri uzatma funksiyası
def extend_text(text, chars=50, temperature=1.0):
    for _ in range(chars):
        text += next_char(text, temperature)  # Hər dəfə növbəti simvolu əlavə edirik
    return text

# Başlanğıc mətni veririk
start_text = "Yenə qılıncını çəkdi üstümə,Qurbanı olduğum o ala gözlər"

# Şeir yaratmaq
generated_text = extend_text(start_text, chars=100, temperature=0.8)

print(generated_text)